In [1]:
import os
import random
import copy
import time
import sys
import shutil
import argparse
import errno
import math
import numpy as np
from collections import defaultdict, OrderedDict
import tensorboardX
from tqdm import tqdm
import pickle as pkl

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

from lib.utils.tools import *
from lib.model.loss import *
from lib.model.loss_mesh import *
from lib.utils.utils_mesh import *
from lib.utils.utils_smpl import *
from lib.utils.utils_data import *
from lib.utils.learning import *
from lib.data.dataset_mesh import MotionSMPL
from lib.model.model_mesh import MeshRegressor
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)

import torch
import numpy as np
import glob
import os
import io
import random
import pickle
from torch.utils.data import Dataset, DataLoader
from lib.data.augmentation import Augmenter3D
from lib.utils.tools import read_pkl
from lib.utils.utils_data import flip_data, crop_scale
from lib.utils.utils_mesh import flip_thetas
from lib.utils.utils_smpl import SMPL
from torch.utils.data import Dataset, DataLoader
from lib.data.datareader_h36m import DataReaderH36M  
from lib.data.datareader_mesh import DataReaderMesh  
from lib.data.dataset_action import random_move  

In [2]:
### THIS SCRIPT IS USED TO EVALUATE THE PREDICTED SMPL VERTICES

In [3]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", type=str, default="configs/pretrain.yaml", help="Path to the config file.")
    parser.add_argument('-c', '--checkpoint', default='checkpoint', type=str, metavar='PATH', help='checkpoint directory')
    parser.add_argument('-p', '--pretrained', default='checkpoint', type=str, metavar='PATH', help='pretrained checkpoint directory')
    parser.add_argument('-r', '--resume', default='', type=str, metavar='FILENAME', help='checkpoint to resume (file name)')
    parser.add_argument('-e', '--evaluate', default='', type=str, metavar='FILENAME', help='checkpoint to evaluate (file name)')
    parser.add_argument('-freq', '--print_freq', default=100)
    parser.add_argument('-ms', '--selection', default='latest_epoch.bin', type=str, metavar='FILENAME', help='checkpoint to finetune (file name)')
    parser.add_argument('-sd', '--seed', default=0, type=int, help='random seed')
    opts = parser.parse_args()
    return opts

In [4]:
with open(r'data/mesh/mesh_det_pw3d.pkl', 'rb') as f:
    dataset_pw3d = pkl.load(f)

In [5]:
dataset_pw3d["train"]['source'][765] # first backpack frame

'courtyard_backpack_000'

In [6]:
dataset_pw3d["test"].keys()

dict_keys(['joint_2d', 'confidence', 'joint_cam', 'smpl_pose', 'smpl_shape', 'img_hw', 'image_id', 'source'])

In [7]:
len(dataset_pw3d['test']['source'])

35515

In [8]:
dataset_pw3d['test']['source'].count('downtown_arguing_001')

895

In [9]:
set(dataset_pw3d['test']['source'])

{'downtown_arguing_000',
 'downtown_arguing_001',
 'downtown_bar_000',
 'downtown_bar_001',
 'downtown_bus_000',
 'downtown_bus_001',
 'downtown_cafe_000',
 'downtown_cafe_001',
 'downtown_car_000',
 'downtown_car_001',
 'downtown_crossStreets_000',
 'downtown_crossStreets_001',
 'downtown_downstairs_000',
 'downtown_enterShop_000',
 'downtown_rampAndStairs_000',
 'downtown_rampAndStairs_001',
 'downtown_runForBus_000',
 'downtown_runForBus_001',
 'downtown_runForBus_010',
 'downtown_runForBus_011',
 'downtown_sitOnStairs_000',
 'downtown_sitOnStairs_001',
 'downtown_stairs_000',
 'downtown_upstairs_000',
 'downtown_walkBridge_010',
 'downtown_walkUphill_000',
 'downtown_walking_000',
 'downtown_walking_001',
 'downtown_warmWelcome_000',
 'downtown_warmWelcome_001',
 'downtown_weeklyMarket_000',
 'downtown_windowShopping_000',
 'flat_guitar_010',
 'flat_packBags_000',
 'office_phoneCall_000',
 'office_phoneCall_001',
 'outdoors_fencing_010'}

In [10]:
# FIGURE OUT WHY length of dataset_pw3d['test']['source'] AND ALL FRAMES FOR PREDICTED VERTICES ARE NOT THE SAME
# I UNDERSTAND IT NOW compose[valid] in /home/philipp/3DPW/readme/predicting.ipynb ias different for every person, so change [iModel] according to which person were looking at 

In [11]:
all_thetas_dt_arguing = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_cafe_00\MotionBert\person_1\vertices_all_frames.npy")

In [12]:
all_thetas_dt_arguing.shape

(934, 6890, 3)

In [13]:
import os
import numpy as np

def count_total_frames(base_path):
    """
    Recursively counts the total number of frames from vertices_all_frames.npy
    files in person_1 and person_2 folders.

    Parameters:
        base_path (str): The root folder path where the search begins.

    Returns:
        int: The total number of frames summed across all files.
    """
    total_frames = 0  # Initialize total frame counter

    # Walk through all subdirectories in the base_path
    for root, dirs, files in os.walk(base_path):
        # Check if we're in a person_1 or person_2 folder
        if os.path.basename(root) in ['person_1', 'person_2']:
            # Check if 'vertices_all_frames.npy' exists in the current folder
            file_path = os.path.join(root, 'vertices_all_frames.npy')
            if os.path.isfile(file_path):
                try:
                    # Load the numpy array and get the number of frames (first dimension)
                    vertices_data = np.load(file_path)
                    nr_frames = vertices_data.shape[0]  # First dimension is number of frames
                    total_frames += nr_frames
                    print(f"Found file: {file_path} | Frames: {nr_frames}")
                except Exception as e:
                    print(f"Error loading file {file_path}: {e}")

    print(f"\nTotal number of frames: {total_frames}")
    return total_frames



# Define the base folder path to search in
base_folder = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"

# Run the frame counting function
total_frames = count_total_frames(base_folder)


Found file: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_1\vertices_all_frames.npy | Frames: 898
Found file: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\vertices_all_frames.npy | Frames: 895
Found file: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_bar_00\MotionBert\person_1\vertices_all_frames.npy | Frames: 1252
Found file: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_bar_00\MotionBert\person_2\vertices_all_frames.npy | Frames: 1206
Found file: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_bus_00\MotionBert\person_1\vertices_all_frames.npy | Frames: 1066
Found file: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_bus_00\Mot

In [14]:
config = 'configs/mesh/MB_train_pw3d.yaml'
args = get_config(config)


trainloader_params = {
        'batch_size': args.batch_size,
        'shuffle': False,
        'num_workers': 0,
        'pin_memory': True,
}
testloader_params = {
        'batch_size': args.batch_size,
        'shuffle': False,
        'num_workers': 0,
        'pin_memory': True,
}

In [15]:

class SMPLDataset(Dataset):
    def __init__(self, args, data_split, dataset): # data_split: train/test; dataset: h36m, coco, pw3d
        random.seed(0)
        np.random.seed(0)
        self.clip_len = args.clip_len
        self.data_split = data_split
    
        if dataset=="h36m":
            datareader = DataReaderH36M(n_frames=self.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=self.clip_len, dt_root=args.data_root, dt_file=args.dt_file_h36m)
        elif dataset=="coco":
            datareader = DataReaderMesh(n_frames=1, sample_stride=args.sample_stride, data_stride_train=1, data_stride_test=1, dt_root=args.data_root, dt_file=args.dt_file_coco, res=[640, 640])
        elif dataset=="pw3d":
            datareader = DataReaderMesh(n_frames=self.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=self.clip_len, dt_root=args.data_root, dt_file=args.dt_file_pw3d, res=[1920, 1920])
        else:
            raise Exception("Mesh dataset undefined.")
        self.datareader = datareader

        split_id_train, split_id_test = datareader.get_split_id()                        # Index of clips
        # print("ID TRAIN:")
        # print(split_id_train)
        # print("ID TEST:")
        # print(split_id_test)
        train_data, test_data = datareader.read_2d()
        train_data, test_data = train_data[split_id_train], test_data[split_id_test]     # Input: (N, T, 17, 3)
        self.motion_2d = {'train': train_data, 'test': test_data}[data_split]

        dt = datareader.dt_dataset
        smpl_pose_train = dt['train']['smpl_pose'][split_id_train]                       # (N, T, 72)
        smpl_shape_train = dt['train']['smpl_shape'][split_id_train]                     # (N, T, 10)
        smpl_pose_test = dt['test']['smpl_pose'][split_id_test]                          # (N, T, 72)
        smpl_shape_test = dt['test']['smpl_shape'][split_id_test]                        # (N, T, 10)
        
        self.motion_smpl_3d = {'train': {'pose': smpl_pose_train, 'shape': smpl_shape_train}, 'test': {'pose': smpl_pose_test, 'shape': smpl_shape_test}}[data_split]
        self.smpl = SMPL(
            args.data_root,
            batch_size=1,
        )
    def get_split_id(self, datareader):
        return datareader.get_split_id()

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.motion_2d)

    def __getitem__(self, index):
        raise NotImplementedError 

class MotionSMPL(SMPLDataset):
    def __init__(self, args, data_split, dataset):
        super(MotionSMPL, self).__init__(args, data_split, dataset)
        self.flip = args.flip
    
        
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        motion_2d = self.motion_2d[index]                                            # motion_2d: (T,17,3)     
        motion_2d[:,:,2] = np.clip(motion_2d[:,:,2], 0, 1)
        motion_smpl_pose = self.motion_smpl_3d['pose'][index].reshape(-1, 24, 3)     # motion_smpl_3d: (T, 24, 3)    
        motion_smpl_shape = self.motion_smpl_3d['shape'][index]                      # motion_smpl_3d: (T,10)    
        
        # if self.data_split=="train":
        #     if self.flip and random.random() > 0.5:                                  # Training augmentation - random flipping
        #         motion_2d = flip_data(motion_2d)
        #         motion_smpl_pose = flip_thetas(motion_smpl_pose)                

            
        motion_smpl_pose = torch.from_numpy(motion_smpl_pose).reshape(-1, 72).float()
        motion_smpl_shape = torch.from_numpy(motion_smpl_shape).reshape(-1, 10).float()
        motion_smpl = self.smpl(
            betas=motion_smpl_shape,
            body_pose=motion_smpl_pose[:, 3:],
            global_orient=motion_smpl_pose[:, :3],
            pose2rot=True
        )
        motion_verts = motion_smpl.vertices.detach()*1000.0
        J_regressor = self.smpl.J_regressor_h36m
        J_regressor_batch = J_regressor[None, :].expand(motion_verts.shape[0], -1, -1).to(motion_verts.device)
        motion_3d_reg = torch.matmul(J_regressor_batch, motion_verts)                 # motion_3d: (T,17,3)  
        motion_verts = motion_verts - motion_3d_reg[:, :1, :]
        motion_3d_reg = motion_3d_reg - motion_3d_reg[:, :1, :]                       # motion_3d: (T,17,3)    
        motion_theta = torch.cat((motion_smpl_pose, motion_smpl_shape), -1)
        motion_smpl_3d = {
            'theta': motion_theta,       # smpl pose and shape
            'kp_3d': motion_3d_reg,      # 3D keypoints
            'verts': motion_verts,       # 3D mesh vertices
        }
        return motion_2d, motion_smpl_3d

In [16]:
mesh_train_pw3d = MotionSMPL(args, data_split='train', dataset="pw3d")
train_loader_pw3d = DataLoader(mesh_train_pw3d, **trainloader_params)

mesh_val_pw3d = MotionSMPL(args, data_split='test', dataset="pw3d")
test_loader_pw3d = DataLoader(mesh_val_pw3d, **testloader_params)

In [17]:
len(mesh_train_pw3d[0][0])

16

In [18]:
len(mesh_val_pw3d)

2202

In [19]:
def main():
    mesh_train_pw3d = MotionSMPL(args, data_split='train', dataset="pw3d")
    train_loader_pw3d = DataLoader(mesh_train_pw3d, **trainloader_params)

    mesh_val_pw3d = MotionSMPL(args, data_split='test', dataset="pw3d")
    test_loader_pw3d = DataLoader(mesh_val_pw3d, **testloader_params)

# Call the main function
main()



In [20]:
split_id_train, split_id_test = mesh_train_pw3d.get_split_id(mesh_train_pw3d.datareader)

In [21]:
# retrieves the ground truth coords for any image 

def get_frame_coords(dataset:str, file_name: str, frame_number: int, split_id_train, mesh_train_pw3d):
    index = None
    resulting_coords = None  # Initialize resulting_coords to avoid potential errors
    
    # Find the first appearance of the file_name
    if dataset == 'train':
        img_names = dataset_pw3d["train"]["source"]
    else:
        img_names = dataset_pw3d["test"]["source"]

    for idx, file in enumerate(img_names):
        if file == file_name:
            index = idx
            break  # Exit the loop as soon as the file is found
    else:
        # This executes if no break occurs in the loop (file not found)
        print("File not found")
        return None, None

    # Calculate the target frame number
    target_number = index + frame_number
    
    # Find the range index
    range_index = next((i for i, r in enumerate(split_id_train) if r.start <= target_number < r.stop), None)
    
    if range_index is not None:
        print(f"The number {target_number} is in range at index {range_index}: {split_id_train[range_index]}")
        # Calculate the index inside the range
        range_start = split_id_train[range_index].start
        index_in_range = target_number - range_start
        
        # Access the batch and ensure indices are valid
        if range_index < len(mesh_train_pw3d) and index_in_range < len(mesh_train_pw3d[range_index][1]['kp_3d']):
            resulting_coords = mesh_train_pw3d[range_index][1]['kp_3d'][index_in_range]
        else:
            print(f"Index {index_in_range} is out of bounds for range index {range_index}.")
    else:
        print(f"The number {target_number} is not in any range of split_id_train.")
        return None, None

    return resulting_coords


In [22]:
resulting_coords = get_frame_coords("test", 'office_phoneCall_00', 98, split_id_test, mesh_val_pw3d)

File not found


In [23]:
#mesh_train_pw3d[batch nr][# keypoints (0), smpl stuff (1)]['kp_3d'][#nr_of frame in the batch]

In [24]:
# import torch
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# # Example tensor of shape 17x3 (replace this with your tensor)
# coordinates = resulting_coords # Replace with your actual tensor

# # Convert tensor to numpy for Matplotlib compatibility
# coordinates_np = coordinates.numpy()

# # Extract X, Y, Z components
# x = coordinates_np[:, 0]
# y = coordinates_np[:, 1]
# z = coordinates_np[:, 2]

# # Create a 3D plot
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# # Scatter plot of the 3D points
# ax.scatter(x, y, z, c='b', marker='o', label="3D Points")

# # Add labels
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.set_title('3D Coordinates Plot')

# # Add a legend
# ax.legend()

# # Show the plot
# plt.show()


In [25]:
#np.save("regressed_coordinates/coordinates_dataset_motionbert.npy", coordinates_np.flatten())

### EVALUATION


In [26]:
# # make predictions 
# #...

# # Mesh regress vertices 
# dataset_pw3d["test"].keys()
# mesh_train_pw3d = MotionSMPL(args, data_split='train', dataset="pw3d")
# mesh_test_pw3d = MotionSMPL(args, data_split='test', dataset="pw3d")

# train_loader_pw3d = DataLoader(mesh_test_pw3d, **trainloader_params)
# test_loader_pw3d = DataLoader(mesh_train_pw3d, **testloader_params)

# J_regressor =np.load("Joint_regressor/J_regressor_h36m_correct.npy")


# # Actually, here, we need to regress the keypoints instead of just using the GT

# predicted_coordinates = mesh_test_pw3d[1][1]

# output = {
#     'theta': None,
#     'verts': None,
#     'kp_3d': None

# }


# # Adding a new dimension to each array
# output['theta'] = np.expand_dims(mesh_test_pw3d[0][1]["theta"].numpy(), axis=0)  # Shape: (1, 16, 72)
# output['verts'] = np.expand_dims(mesh_test_pw3d[0][1]["verts"].numpy(), axis=0)  # Shape: (1, 16, 6890, 3)
# output['kp_3d'] = np.expand_dims(mesh_test_pw3d[0][1]["kp_3d"].numpy(), axis=0)  # Shape: (1, 16, 17, 3)





In [27]:
batch_count = 0

for batch in test_loader_pw3d:
    batch_count += 1
    
print(f"Total number of batches: {batch_count}")

Total number of batches: 18


In [28]:
for idx, (batch_input, batch_gt) in tqdm(enumerate(test_loader_pw3d)):
    batch_size, clip_len = batch_input.shape[:2]
    if torch.cuda.is_available():
        batch_gt['theta'] = batch_gt['theta'].cuda().float()
        batch_gt['kp_3d'] = batch_gt['kp_3d'].cuda().float()
        batch_gt['verts'] = batch_gt['verts'].cuda().float()
        batch_input = batch_input.cuda().float()

18it [00:25,  1.41s/it]


In [29]:
batch_gt['theta'].shape

torch.Size([26, 16, 82])

In [30]:
 
theta = torch.randn(batch_gt['theta'].shape[0], batch_gt['theta'].shape[1], batch_gt['theta'].shape[2])
kp_3d = torch.randn(batch_gt['kp_3d'].shape[0], batch_gt['kp_3d'].shape[0], batch_gt['kp_3d'].shape[0], batch_gt['kp_3d'].shape[0])
verts = torch.randn(batch_gt['verts'].shape[0], batch_gt['verts'].shape[0], batch_gt['verts'].shape[0], batch_gt['verts'].shape[0])

# Create the dictionary
sample_prediction = {
    "theta": theta,
    "kp_3d": kp_3d,
    "verts": verts,
}

In [31]:
print(theta.shape, kp_3d.shape, verts.shape)

torch.Size([26, 16, 82]) torch.Size([26, 26, 26, 26]) torch.Size([26, 26, 26, 26])


In [32]:
# "output is a dict of shape"
# theta: (1, T, dim_theta) (e.g., (1, T, 72))
# verts: (1, T, num_verts, 3) (e.g., (1, T, 6890, 3) if num_verts=6890)
# kp_3d: (1, T, 17, 3)



def validate(test_loader, output, criterion, dataset_name='h36m'):

    print(f'===========> validating {dataset_name}')
    batch_time = AverageMeter()
    losses = AverageMeter()
    losses_dict = {'loss_3d_pos': AverageMeter(), 
                   'loss_3d_scale': AverageMeter(), 
                   'loss_3d_velocity': AverageMeter(),
                   'loss_lv': AverageMeter(), 
                   'loss_lg': AverageMeter(), 
                   'loss_a': AverageMeter(), 
                   'loss_av': AverageMeter(), 
                   'loss_pose': AverageMeter(), 
                   'loss_shape': AverageMeter(),
                   'loss_norm': AverageMeter(),
    }
    mpjpes = AverageMeter()
    mpves = AverageMeter()
    results = defaultdict(list)
    smpl = SMPL(args.data_root, batch_size=1).cuda()
    J_regressor = smpl.J_regressor_h36m
    with torch.no_grad():
        end = time.time()
        for idx, (batch_input, batch_gt) in tqdm(enumerate(test_loader)):
            batch_size, clip_len = batch_input.shape[:2]
            if torch.cuda.is_available():
                batch_gt['theta'] = batch_gt['theta'].cuda().float()
                batch_gt['kp_3d'] = batch_gt['kp_3d'].cuda().float()
                batch_gt['verts'] = batch_gt['verts'].cuda().float()
                batch_input = batch_input.cuda().float()

            theta = torch.randn(batch_gt['theta'].shape[0], batch_gt['theta'].shape[1], batch_gt['theta'].shape[2])
            kp_3d = torch.randn(batch_gt['kp_3d'].shape[0], batch_gt['kp_3d'].shape[1], batch_gt['kp_3d'].shape[2], batch_gt['kp_3d'].shape[3])
            verts = torch.randn(batch_gt['verts'].shape[0], batch_gt['verts'].shape[1], batch_gt['verts'].shape[2], batch_gt['verts'].shape[3])

            theta = theta.to('cuda')
            kp_3d = kp_3d.to('cuda')
            verts = verts.to('cuda')



            # Create the dictionary
            sample_prediction = {
                "theta": theta,
                "kp_3d": kp_3d,
                "verts": verts,
            }
            
            output = [sample_prediction]

            output_final = output
            
            print('Pred is')
            print(output[0]['kp_3d'].shape)

            print('GT is')
            print(batch_gt['kp_3d'].shape)

            loss_dict = criterion(output, batch_gt)
            loss = args.lambda_3d      * loss_dict['loss_3d_pos']      + \
                   args.lambda_scale   * loss_dict['loss_3d_scale']    + \
                   args.lambda_3dv     * loss_dict['loss_3d_velocity'] + \
                   args.lambda_lv      * loss_dict['loss_lv']          + \
                   args.lambda_lg      * loss_dict['loss_lg']          + \
                   args.lambda_a       * loss_dict['loss_a']           + \
                   args.lambda_av      * loss_dict['loss_av']          + \
                   args.lambda_shape   * loss_dict['loss_shape']       + \
                   args.lambda_pose    * loss_dict['loss_pose']        + \
                   args.lambda_norm    * loss_dict['loss_norm'] 
            # update metric
            losses.update(loss.item(), batch_size)
            loss_str = ''
            for k, v in loss_dict.items():
                losses_dict[k].update(v.item(), batch_size)
                loss_str += '{0} {loss.val:.3f} ({loss.avg:.3f})\t'.format(k, loss=losses_dict[k])
            mpjpe, mpve = compute_error(output, batch_gt)
            mpjpes.update(mpjpe, batch_size)
            mpves.update(mpve, batch_size)
            
            for keys in output[0].keys():
                if type(output[0][keys]) != np.ndarray: 
                    output[0][keys] = output[0][keys].detach().cpu().numpy()
                if type(batch_gt[keys]) != np.ndarray: 
                    batch_gt[keys] = batch_gt[keys].detach().cpu().numpy()
            results['kp_3d'].append(output[0]['kp_3d'])
            results['verts'].append(output[0]['verts'])
            results['kp_3d_gt'].append(batch_gt['kp_3d'])
            results['verts_gt'].append(batch_gt['verts'])

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if idx % int(100) == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      '{2}'
                      'PVE {mpves.val:.3f} ({mpves.avg:.3f})\t'
                      'JPE {mpjpes.val:.3f} ({mpjpes.avg:.3f})'.format(
                       idx, len(test_loader), loss_str, batch_time=batch_time,
                       loss=losses, mpves=mpves, mpjpes=mpjpes))
            

    print(f'==> start concating results of {dataset_name}')
    for term in results.keys():
        results[term] = np.concatenate(results[term])
    print(f'==> start evaluating {dataset_name}...')
    error_dict = evaluate_mesh(results)
    err_str = ''
    for err_key, err_val in error_dict.items():
        err_str += '{}: {:.2f}mm \t'.format(err_key, err_val)
    print(f'=======================> {dataset_name} validation done: ', loss_str)
    print(f'=======================> {dataset_name} validation done: ', err_str)
    return losses.avg, error_dict['mpjpe'], error_dict['pa_mpjpe'], error_dict['mpve'], losses_dict

In [33]:
criterion = MeshLoss(loss_type = args.loss_type)

In [34]:
# if __name__ == "__main__":
#     mesh_train_pw3d = MotionSMPL(args, data_split='train', dataset="pw3d")
#     mesh_test_pw3d = MotionSMPL(args, data_split='test', dataset="pw3d")

#     train_loader_pw3d = DataLoader(mesh_train_pw3d, **trainloader_params)
#     test_loader_pw3d = DataLoader(mesh_test_pw3d, **testloader_params)




test_loss, test_mpjpe, test_pa_mpjpe, test_mpve, test_losses_dict = validate(test_loader_pw3d, output=None, criterion=criterion, dataset_name='pw3d')

===========> validating pw3d


0it [00:00, ?it/s]

Pred is
torch.Size([128, 16, 17, 3])
GT is
torch.Size([128, 16, 17, 3])


1it [00:01,  1.99s/it]

Test: [0/18]	Time 1.988 (1.988)	Loss 1009.5448 (1009.5448)	loss_3d_pos 407.695 (407.695)	loss_3d_scale 404.192 (404.192)	loss_3d_velocity 6.694 (6.694)	loss_lv 0.908 (0.908)	loss_lg 239.669 (239.669)	loss_a 1.201 (1.201)	loss_av 0.685 (0.685)	loss_shape 0.984 (0.984)	loss_pose 0.557 (0.557)	loss_norm 9.016 (9.016)	PVE 455.068 (455.068)	JPE 407.695 (407.695)


...

17it [00:30,  1.82s/it]

Pred is
torch.Size([128, 16, 17, 3])
GT is
torch.Size([128, 16, 17, 3])


18it [00:31,  1.73s/it]

Pred is
torch.Size([26, 16, 17, 3])
GT is
torch.Size([26, 16, 17, 3])
==> start concating results of pw3d


==> start evaluating pw3d...
=======================> pw3d validation done:  loss_3d_pos 410.785 (425.226)	loss_3d_scale 406.593 (419.881)	loss_3d_velocity 5.649 (10.397)	loss_lv 0.938 (0.907)	loss_lg 250.634 (254.918)	loss_a 1.191 (1.169)	loss_av 0.693 (0.687)	loss_shape 1.086 (1.070)	loss_pose 0.561 (0.561)	loss_norm 9.113 (9.028)	
=======================> pw3d validation done:  mpve: 472.44mm 	mpjpe: 458.91mm 	pa_mpjpe: 416.31mm 	mpjpe_17j: 425.23mm 	pa_mpjpe_17j: 393.10mm 	


In [35]:
smpl = SMPL(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\data\mesh", batch_size=1).cuda()
J_regressor = smpl.J_regressor_h36m

J_regressor_motionbert = np.load("Joint_regressor/J_regressor_h36m_correct.npy")

In [36]:
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# # Ground Truth joint coordinates

# # Reshape the list into a 24x3 array
# coordinates = np.array(joint_positions_gt).reshape(24, 3)

# # Extract x, y, z coordinates
# x = coordinates[:, 0]
# y = coordinates[:, 1]
# z = coordinates[:, 2]

# # Plot the 3D points
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(x, y, z, c='r', marker='o', label='Joint Positions')

# # Optional: Add labels and a legend
# ax.set_xlabel('X Coordinate')
# ax.set_ylabel('Y Coordinate')
# ax.set_zlabel('Z Coordinate')
# ax.set_title('3D Joint Positions')
# ax.legend()

# # Show the plot
# plt.show()


In [37]:
# # Regressed coords for GT SMPL and for predicted SMPL

# GT = False

# if GT:
#     # Extract X, Y, Z coordinates
#     x_coords = result_gt[:, 0]
#     y_coords = result_gt[:, 1]
#     z_coords = result_gt[:, 2]
# else:
#     # Extract X, Y, Z coordinates
#     x_coords = result_pred[:, 0]
#     y_coords = result_pred[:, 1]
#     z_coords = result_pred[:, 2]

# # Plot in 3D
# fig = plt.figure(figsize=(8, 6))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(x_coords, y_coords, z_coords, c='r', marker='o', label="Joints")

# # Label the axes
# ax.set_xlabel('X Coordinate')
# ax.set_ylabel('Y Coordinate')
# ax.set_zlabel('Z Coordinate')

# # Add a title and legend
# ax.set_title("3D Joint Positions")
# ax.legend()

# # Show the plot
# plt.show()

In [38]:
# np.save("Regressed_Coordinates/regressed_backpack_pred_00.npy",result_pred.flatten())
# #Currently finding out what the difference is between the original ground truth keypoints, and the keypoints I get when I regress the ground truth SMPL vertices set

In [39]:
#TODO: Take vertices ground truth of backpack first frame and regress here. Then plot it 

### Testing backpack example

In [40]:
J_regressor_motionbert = np.load("Joint_regressor/J_regressor_h36m_correct.npy")


def thetas_to_kp_3d(pred_thetas, pred_verts, J_regressor):
    pred_thetas = np.concatenate([pred_thetas, np.zeros(10)])
    pred_kp_3d = J_regressor @ pred_verts


    # Define the rotation matrix for -180 degrees around the x-axis
    R_x = np.array([
        [1,  0,  0],
        [0, -1,  0],
        [0,  0, -1]
    ])

    # Rotate each point by applying the rotation matrix
    pred_kp_3d = pred_kp_3d @ R_x.T

    

In [41]:
pred_thetas = np.load('Prediction_necessities/all_thetas_backpack.npy')[0]
pred_thetas = np.concatenate([pred_thetas, np.zeros(10)])
pred_verts = np.load('Prediction_necessities/vertices_backpack_frame_00_pred.npy')
#J_regressor = J_regressor.to('cpu')
pred_kp_3d = J_regressor_motionbert @ pred_verts


In [42]:
# Define the rotation matrix for -180 degrees around the x-axis
R_x = np.array([
    [1,  0,  0],
    [0, -1,  0],
    [0,  0, -1]
])

# Rotate each point by applying the rotation matrix
pred_kp_3d = pred_kp_3d @ R_x.T


In [43]:
# Extract the first coordinate
first_point = pred_kp_3d[0]

# Translate all points to set the first point as the origin
pred_kp_3d = pred_kp_3d - first_point

# Ensure the first point is now exactly zero
pred_kp_3d[0] = np.array([0.0, 0.0, 0.0])

In [44]:
import torch

pred_thetas = torch.tensor(pred_thetas).unsqueeze(0).unsqueeze(0)
pred_verts = torch.tensor(pred_verts).unsqueeze(0).unsqueeze(0)
pred_kp_3d = torch.tensor(pred_kp_3d).unsqueeze(0).unsqueeze(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move tensors to GPU
pred_thetas = pred_thetas.to(device)
pred_verts = pred_verts.to(device)
pred_kp_3d = pred_kp_3d.to(device)



In [45]:
sample_prediction = {
    "theta": pred_thetas,
    "kp_3d": pred_kp_3d,
    "verts": pred_verts,
}

In [46]:
pred_kp_3d.shape

torch.Size([1, 1, 17, 3])

In [47]:
# resulting_coords = get_frame_coords("train", 'courtyard_backpack_000', 0, split_id_train, mesh_train_pw3d)

In [48]:
# np.save('Regressed_Coordinates/regressed_backpack_origin.npy', (sample_prediction['kp_3d']*1000).flatten())

In [49]:
#np.save('Regressed_Coordinates/ground_truth_coords.npy', resulting_coords.flatten())

In [50]:
batch = next(iter(train_loader_pw3d))

In [51]:
# These two are the same 
mesh_train_pw3d[94][1]['kp_3d'][0]
batch[1]['kp_3d'][94][0]

tensor([[   0.0000,    0.0000,    0.0000],
        [-140.4294,    1.5662,  -45.2371],
        [ -75.8165,  430.4894,   21.4302],
        [ -97.8810,  825.3510,  165.0902],
        [ 139.2287,   -0.9688,   44.7472],
        [ 117.8244,  438.8577,   61.3678],
        [  88.7600,  817.7072,  217.9599],
        [ -12.2836, -249.0143,   -9.4596],
        [   3.7105, -480.3842,  -85.9170],
        [  36.8778, -508.3334, -181.2408],
        [  43.1729, -616.3802, -216.4468],
        [ 133.6753, -435.5518,  -16.6594],
        [ 223.1522, -189.5069,   75.7827],
        [ 195.9183,   58.6210,   92.5941],
        [-136.4164, -425.4830, -106.7033],
        [-237.8101, -171.6483,  -67.3985],
        [-198.9982,   70.9759,  -42.6763]])

In [52]:
get_frame_coords("train", 'courtyard_backpack_000', 0, split_id_train, mesh_train_pw3d)

The number 765 is in range at index 94: range(765, 781)


tensor([[   0.0000,    0.0000,    0.0000],
        [-140.4294,    1.5662,  -45.2371],
        [ -75.8165,  430.4894,   21.4302],
        [ -97.8810,  825.3510,  165.0902],
        [ 139.2287,   -0.9688,   44.7472],
        [ 117.8244,  438.8577,   61.3678],
        [  88.7600,  817.7072,  217.9599],
        [ -12.2836, -249.0143,   -9.4596],
        [   3.7105, -480.3842,  -85.9170],
        [  36.8778, -508.3334, -181.2408],
        [  43.1729, -616.3802, -216.4468],
        [ 133.6753, -435.5518,  -16.6594],
        [ 223.1522, -189.5069,   75.7827],
        [ 195.9183,   58.6210,   92.5941],
        [-136.4164, -425.4830, -106.7033],
        [-237.8101, -171.6483,  -67.3985],
        [-198.9982,   70.9759,  -42.6763]])

In [53]:
sample_prediction["theta"].shape

torch.Size([1, 1, 82])

In [54]:
# Extract, reshape, and convert batch[0]
batch[0] = batch[0][0][0]
batch[0] = torch.tensor(batch[0]).unsqueeze(0).unsqueeze(0).to(device)

# Extract, reshape, and convert batch[1] elements
batch[1]['theta'] = torch.tensor(batch[1]['theta'][0][0]).unsqueeze(0).unsqueeze(0).to(device)
batch[1]['kp_3d'] = torch.tensor(batch[1]['kp_3d'][0][0]).unsqueeze(0).unsqueeze(0).to(device)
batch[1]['verts'] = torch.tensor(batch[1]['verts'][0][0]).unsqueeze(0).unsqueeze(0).to(device)

e:\miniconda-envs\motionbert\lib\site-packages\ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
e:\miniconda-envs\motionbert\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
e:\miniconda-envs\motionbert\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
e:\miniconda-envs\motionbert\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy const

In [55]:
batch[1]['kp_3d'].shape

torch.Size([1, 1, 17, 3])

In [56]:
batch[0].shape

torch.Size([1, 1, 17, 3])

In [57]:
# "output is a dict of shape"
# theta: (1, T, dim_theta) (e.g., (1, T, 72))
# verts: (1, T, num_verts, 3) (e.g., (1, T, 6890, 3) if num_verts=6890)
# kp_3d: (1, T, 17, 3)



def validate(criterion, dataset_name='h36m'):

    print(f'===========> validating {dataset_name}')
    batch_time = AverageMeter()
    losses = AverageMeter()
    losses_dict = {'loss_3d_pos': AverageMeter(), 
                   'loss_3d_scale': AverageMeter(), 
                   'loss_3d_velocity': AverageMeter(),
                   'loss_lv': AverageMeter(), 
                   'loss_lg': AverageMeter(), 
                   'loss_a': AverageMeter(), 
                   'loss_av': AverageMeter(), 
                   'loss_pose': AverageMeter(), 
                   'loss_shape': AverageMeter(),
                   'loss_norm': AverageMeter(),
    }
    mpjpes = AverageMeter()
    mpves = AverageMeter()
    results = defaultdict(list)
    smpl = SMPL(args.data_root, batch_size=1).cuda()
    J_regressor = smpl.J_regressor_h36m
    with torch.no_grad():
        end = time.time()
    ######################################################################
    idx = 0
    batch_input = batch[0]
    batch_gt = batch[1]

    batch_size, clip_len = batch_input.shape[:2]
    if torch.cuda.is_available():
        batch_gt['theta'] = batch_gt['theta'].cuda().float()
        batch_gt['kp_3d'] = batch_gt['kp_3d'].cuda().float()
        batch_gt['verts'] = batch_gt['verts'].cuda().float()
        batch_input = batch_input.cuda().float()

    output = [sample_prediction]

    output_final = output

    print('Pred is')
    print(output[0]['kp_3d'].shape)

    print('GT is')
    print(batch_gt['kp_3d'].shape)


    loss_dict = criterion(output, batch_gt)
    
    loss = args.lambda_3d      * loss_dict['loss_3d_pos']      + \
        args.lambda_scale   * loss_dict['loss_3d_scale']    + \
        args.lambda_3dv     * loss_dict['loss_3d_velocity'] + \
        args.lambda_lv      * loss_dict['loss_lv']          + \
        args.lambda_lg      * loss_dict['loss_lg']          + \
        args.lambda_a       * loss_dict['loss_a']           + \
        args.lambda_av      * loss_dict['loss_av']          + \
        args.lambda_shape   * loss_dict['loss_shape']       + \
        args.lambda_pose    * loss_dict['loss_pose']        + \
        args.lambda_norm    * loss_dict['loss_norm'] 
    # update metric
    losses.update(loss.item(), batch_size)
    loss_str = ''
    for k, v in loss_dict.items():
        losses_dict[k].update(v.item(), batch_size)
        loss_str += '{0} {loss.val:.3f} ({loss.avg:.3f})\t'.format(k, loss=losses_dict[k])
    mpjpe, mpve = compute_error(output, batch_gt)
    mpjpes.update(mpjpe, batch_size)
    mpves.update(mpve, batch_size)
    
    for keys in output[0].keys():
        if type(output[0][keys]) != np.ndarray: 
            output[0][keys] = output[0][keys].detach().cpu().numpy()
        if type(batch_gt[keys]) != np.ndarray: 
            batch_gt[keys] = batch_gt[keys].detach().cpu().numpy()
    results['kp_3d'].append(output[0]['kp_3d'])
    results['verts'].append(output[0]['verts'])
    results['kp_3d_gt'].append(batch_gt['kp_3d'])
    results['verts_gt'].append(batch_gt['verts'])

    # measure elapsed time
    batch_time.update(time.time() - end)
    end = time.time()

    if idx % int(100) == 0:
        print('Test: [{0}/{1}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
            '{2}'
            'PVE {mpves.val:.3f} ({mpves.avg:.3f})\t'
            'JPE {mpjpes.val:.3f} ({mpjpes.avg:.3f})'.format(
            idx, 1, loss_str, batch_time=batch_time,
            loss=losses, mpves=mpves, mpjpes=mpjpes))
           ###################################################################

    print(f'==> start concating results of {dataset_name}')
    for term in results.keys():
        results[term] = np.concatenate(results[term])
    print(f'==> start evaluating {dataset_name}...')
    error_dict = evaluate_mesh(results)
    err_str = ''
    for err_key, err_val in error_dict.items():
        err_str += '{}: {:.2f}mm \t'.format(err_key, err_val)
    print(f'=======================> {dataset_name} validation done: ', loss_str)
    print(f'=======================> {dataset_name} validation done: ', err_str)
    return losses.avg, error_dict['mpjpe'], error_dict['pa_mpjpe'], error_dict['mpve'], losses_dict

In [58]:
128, 16, 17, 3

(128, 16, 17, 3)

In [59]:
test_loss, test_mpjpe, test_pa_mpjpe, test_mpve, test_losses_dict = validate(criterion=criterion, dataset_name='3dpw')

===========> validating 3dpw
Pred is
torch.Size([1, 1, 17, 3])
GT is
torch.Size([1, 1, 17, 3])
Test: [0/1]	Time 0.020 (0.020)	Loss 341.7729 (341.7729)	loss_3d_pos 382.523 (382.523)	loss_3d_scale 89.297 (89.297)	loss_3d_velocity 0.000 (0.000)	loss_lv 0.000 (0.000)	loss_lg 245.121 (245.121)	loss_a 0.109 (0.109)	loss_av 0.000 (0.000)	loss_shape 0.302 (0.302)	loss_pose 0.103 (0.103)	loss_norm 2.350 (2.350)	PVE 412.270 (412.270)	JPE 382.523 (382.523)
==> start concating results of 3dpw
==> start evaluating 3dpw...
=======================> 3dpw validation done:  loss_3d_pos 382.523 (382.523)	loss_3d_scale 89.297 (89.297)	loss_3d_velocity 0.000 (0.000)	loss_lv 0.000 (0.000)	loss_lg 245.121 (245.121)	loss_a 0.109 (0.109)	loss_av 0.000 (0.000)	loss_shape 0.302 (0.302)	loss_pose 0.103 (0.103)	loss_norm 2.350 (2.350)	
=======================> 3dpw validation done:  mpve: 412.27mm 	mpjpe: 408.34mm 	pa_mpjpe: 44.19mm 	mpjpe_17j: 382.52mm 	pa_mpjpe_17j: 43.20mm 	


In [60]:
# Next: pipeline to predict all images and then get values (test set)

In [61]:
# Output should be a list of these

sample_prediction = {
    "theta": pred_thetas,
    "kp_3d": pred_kp_3d,
    "verts": pred_verts,
}

# Afterwards, check if they have the sae order as the GT

In [62]:
all_verts = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_1\all_thetas.npy")

In [63]:
all_verts.shape

(898, 72)

In [78]:
## CALCULATE ALL EVALUATION DICTIONARIES FOR ONE PERSON OF A SEQUENCE



J_regressor_motionbert = np.load("Joint_regressor/J_regressor_h36m_correct.npy")




all_pred_thetas = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_1\all_thetas.npy")
#all_pred_thetas = np.concatenate([pred_thetas, np.zeros(10)])
all_pred_verts = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_1\vertices_all_frames.npy")
#J_regressor = J_regressor.to('cpu')
#all_pred_kp_3d = J_regressor_motionbert @ pred_verts



def preds_to_dict(all_pred_thetas, all_pred_verts, J_regressor):
    all_outputs = []
    
    for i in range(len(all_pred_thetas)-1):
        pred_thetas = all_pred_thetas[i]
        pred_verts = all_pred_verts[i]

        pred_thetas = np.concatenate([pred_thetas, np.zeros(10)])
        pred_kp_3d = J_regressor @ pred_verts


        # Define the rotation matrix for -180 degrees around the x-axis
        R_x = np.array([
            [1,  0,  0],
            [0, -1,  0],
            [0,  0, -1]
        ])

        # Rotate each point by applying the rotation matrix
        pred_kp_3d = pred_kp_3d @ R_x.T


        # Extract the first coordinate
        first_point = pred_kp_3d[0]

        # Translate all points to set the first point as the origin
        pred_kp_3d = pred_kp_3d - first_point

        # Ensure the first point is now exactly zero
        pred_kp_3d[0] = np.array([0.0, 0.0, 0.0])


        pred_thetas = torch.tensor(pred_thetas).unsqueeze(0).unsqueeze(0)
        pred_verts = torch.tensor(pred_verts).unsqueeze(0).unsqueeze(0)
        pred_kp_3d = torch.tensor(pred_kp_3d).unsqueeze(0).unsqueeze(0)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Move tensors to GPU
        pred_thetas = pred_thetas.to(device)
        pred_verts = pred_verts.to(device)
        pred_kp_3d = pred_kp_3d.to(device)


        sample_prediction = {
            "theta": pred_thetas,
            "kp_3d": pred_kp_3d,
            "verts": pred_verts,
        }

        all_outputs.append(sample_prediction)

    return(all_outputs)



In [65]:
all_sequence_outputs = preds_to_dict(all_pred_thetas, all_pred_verts, J_regressor_motionbert)

In [80]:
all_pred_thetas = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\all_thetas.npy")
#all_pred_thetas = np.concatenate([pred_thetas, np.zeros(10)])
all_pred_verts = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\vertices_all_frames.npy")

In [82]:
all_pred_verts.shape

(895, 6890, 3)

In [ ]:
## LOOK AT CREATION OF THETAS; ONLY SAVE THE ONES WITH COMPOSE_VALID

In [79]:
base_dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"
all_sequences_outputs = []

# Walk through the directory structure
for root, dirs, files in os.walk(base_dir):
    if "MotionBert" in root:
        person_folders = [os.path.join(root, d) for d in dirs if d.startswith("person_")]
        for person_folder in person_folders:
            # Load the files
            vertices_path = os.path.join(person_folder, "vertices_all_frames.npy")
            thetas_path = os.path.join(person_folder, "all_thetas.npy")
            print(thetas_path)
            
            if os.path.exists(vertices_path) and os.path.exists(thetas_path):
                all_pred_verts = np.load(vertices_path)
                all_pred_thetas = np.load(thetas_path)
                
                # Call the function
                sequence_output = preds_to_dict(all_pred_thetas, all_pred_verts, J_regressor_motionbert)
                all_sequences_outputs.append(sequence_output)

E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_1\all_thetas.npy
E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\all_thetas.npy


IndexError: index 895 is out of bounds for axis 0 with size 895

In [45]:
import os
import numpy as np

# Base directory
base_dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"

# To store all the results
all_outputs = []

# Walk through the directory structure
for root, dirs, files in os.walk(base_dir):
    if "MotionBert" in root:
        person_folders = [os.path.join(root, d) for d in dirs if d.startswith("person_")]
        for person_folder in person_folders:
            # Load the files
            vertices_path = os.path.join(person_folder, "vertices_all_frames.npy")
            thetas_path = os.path.join(person_folder, "all_thetas.npy")
            
            if os.path.exists(vertices_path) and os.path.exists(thetas_path):
                print(vertices_path)
                pred_verts = np.load(vertices_path)
                pred_thetas = np.load(thetas_path)
                
                # Call the function
                sample_prediction = preds_to_dict(pred_thetas, pred_verts, J_regressor_motionbert)
                all_outputs.append(sample_prediction)

# Now `all_outputs` contains all the processed predictions
print(f"Processed {len(all_outputs)} outputs.")


E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_1\vertices_all_frames.npy


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

## WHOLE DATASET

In [ ]:
def validate(test_loader, model, criterion, dataset_name='h36m'):
    model.eval()
    print(f'===========> validating {dataset_name}')
    batch_time = AverageMeter()
    losses = AverageMeter()
    losses_dict = {'loss_3d_pos': AverageMeter(), 
                   'loss_3d_scale': AverageMeter(), 
                   'loss_3d_velocity': AverageMeter(),
                   'loss_lv': AverageMeter(), 
                   'loss_lg': AverageMeter(), 
                   'loss_a': AverageMeter(), 
                   'loss_av': AverageMeter(), 
                   'loss_pose': AverageMeter(), 
                   'loss_shape': AverageMeter(),
                   'loss_norm': AverageMeter(),
    }
    mpjpes = AverageMeter()
    mpves = AverageMeter()
    results = defaultdict(list)
    smpl = SMPL(args.data_root, batch_size=1).cuda()
    J_regressor = smpl.J_regressor_h36m
    with torch.no_grad():
        end = time.time()
        for idx, (batch_input, batch_gt) in tqdm(enumerate(test_loader)):
            batch_size, clip_len = batch_input.shape[:2]
            if torch.cuda.is_available():
                batch_gt['theta'] = batch_gt['theta'].cuda().float()
                batch_gt['kp_3d'] = batch_gt['kp_3d'].cuda().float()
                batch_gt['verts'] = batch_gt['verts'].cuda().float()
                batch_input = batch_input.cuda().float()
            output = model(batch_input)    
            output_final = output
            if args.flip:
                batch_input_flip = flip_data(batch_input)
                output_flip = model(batch_input_flip)
                output_flip_pose = output_flip[0]['theta'][:, :, :72]
                output_flip_shape = output_flip[0]['theta'][:, :, 72:]
                output_flip_pose = flip_thetas_batch(output_flip_pose)
                output_flip_pose = output_flip_pose.reshape(-1, 72)
                output_flip_shape = output_flip_shape.reshape(-1, 10)
                output_flip_smpl = smpl(
                    betas=output_flip_shape,
                    body_pose=output_flip_pose[:, 3:],
                    global_orient=output_flip_pose[:, :3],
                    pose2rot=True
                )
                output_flip_verts = output_flip_smpl.vertices.detach()*1000.0
                J_regressor_batch = J_regressor[None, :].expand(output_flip_verts.shape[0], -1, -1).to(output_flip_verts.device)
                output_flip_kp3d = torch.matmul(J_regressor_batch, output_flip_verts)  # (NT,17,3) 
                output_flip_back = [{
                    'theta': torch.cat((output_flip_pose.reshape(batch_size, clip_len, -1), output_flip_shape.reshape(batch_size, clip_len, -1)), dim=-1),
                    'verts': output_flip_verts.reshape(batch_size, clip_len, -1, 3),
                    'kp_3d': output_flip_kp3d.reshape(batch_size, clip_len, -1, 3),
                }]
                output_final = [{}]
                for k, v in output_flip[0].items():
                    output_final[0][k] = (output[0][k] + output_flip_back[0][k])*0.5
                output = output_final
            loss_dict = criterion(output, batch_gt)
            loss = args.lambda_3d      * loss_dict['loss_3d_pos']      + \
                   args.lambda_scale   * loss_dict['loss_3d_scale']    + \
                   args.lambda_3dv     * loss_dict['loss_3d_velocity'] + \
                   args.lambda_lv      * loss_dict['loss_lv']          + \
                   args.lambda_lg      * loss_dict['loss_lg']          + \
                   args.lambda_a       * loss_dict['loss_a']           + \
                   args.lambda_av      * loss_dict['loss_av']          + \
                   args.lambda_shape   * loss_dict['loss_shape']       + \
                   args.lambda_pose    * loss_dict['loss_pose']        + \
                   args.lambda_norm    * loss_dict['loss_norm'] 
            # update metric
            losses.update(loss.item(), batch_size)
            loss_str = ''
            for k, v in loss_dict.items():
                losses_dict[k].update(v.item(), batch_size)
                loss_str += '{0} {loss.val:.3f} ({loss.avg:.3f})\t'.format(k, loss=losses_dict[k])
            mpjpe, mpve = compute_error(output, batch_gt)
            mpjpes.update(mpjpe, batch_size)
            mpves.update(mpve, batch_size)
            
            for keys in output[0].keys():
                output[0][keys] = output[0][keys].detach().cpu().numpy()
                batch_gt[keys] = batch_gt[keys].detach().cpu().numpy()
            results['kp_3d'].append(output[0]['kp_3d'])
            results['verts'].append(output[0]['verts'])
            results['kp_3d_gt'].append(batch_gt['kp_3d'])
            results['verts_gt'].append(batch_gt['verts'])

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if idx % int(100) == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      '{2}'
                      'PVE {mpves.val:.3f} ({mpves.avg:.3f})\t'
                      'JPE {mpjpes.val:.3f} ({mpjpes.avg:.3f})'.format(
                       idx, len(test_loader), loss_str, batch_time=batch_time,
                       loss=losses, mpves=mpves, mpjpes=mpjpes))

    print(f'==> start concating results of {dataset_name}')
    for term in results.keys():
        results[term] = np.concatenate(results[term])
    print(f'==> start evaluating {dataset_name}...')
    error_dict = evaluate_mesh(results)
    err_str = ''
    for err_key, err_val in error_dict.items():
        err_str += '{}: {:.2f}mm \t'.format(err_key, err_val)
    print(f'=======================> {dataset_name} validation done: ', loss_str)
    print(f'=======================> {dataset_name} validation done: ', err_str)
    return losses.avg, error_dict['mpjpe'], error_dict['pa_mpjpe'], error_dict['mpve'], losses_dict